In [1]:
# Uncomment, if you need to install Pathway and OpenAI packages
!pip install pathway
!pip install mistralai
# Download `adaptive-rag-contexts.jsonl` with ~1000 contexts from SQUAD dataset
!wget -q -nc https://public-pathway-releases.s3.eu-central-1.amazonaws.com/data/adaptive-rag-contexts.jsonl

# If you want to use cache, set `PATHWAY_PERSISTENT_STORAGE environmental variable
# !export PATHWAY_PERSISTENT_STORAGE=".cache"

In [ ]:
# TO DO
# 1. use LiteLLM Embedder/Chat
# 2. Forward to Mistral Cloud API
# 3. Test
# 4. Write local version with SentenceTransformers
# 5. Test
# 6. Write text
# 7. Submit PR

In [2]:
import getpass
import os

import pandas as pd

import pathway as pw
from pathway.stdlib.indexing import VectorDocumentIndex
from pathway.xpacks.llm.embedders import LiteLLMEmbedder, SentenceTransformerEmbedder #OpenAIEmbedder
from pathway.xpacks.llm.llms import LiteLLMChat #OpenAIChat 
from pathway.xpacks.llm.question_answering import (
    answer_with_geometric_rag_strategy_from_index,
)

In [2]:
# Check API key
if "MISTRAL_API_KEY" in os.environ:
    mistral_api_key = os.environ["MISTRAL_API_KEY"]
else:
    mistral_api_key = getpass.getpass("Mistral API Key:")

Mistral API Key: ········


In [3]:
# Flags for running locally

run_local = "No"
local_llm = "mistral:instruct"

In [5]:
# Embed and index
embedding_dimension: int = 1024

if run_local == "Yes":
    pass
    #embedding = GPT4AllEmbeddings()
else:
    embedder = LiteLLMEmbedder(
        capacity = 5, 
        retry_strategy = pw.udfs.FixedDelayRetryStrategy(),
        model = "mistral/mistral-embed",
        api_key=mistral_api_key,
    )

In [6]:
# Load documents in which answers will be searched
class InputSchema(pw.Schema):
    doc: str


documents = pw.io.fs.read(
    "adaptive-rag-contexts.jsonl",
    format="json",
    schema=InputSchema,
    json_field_paths={"doc": "/context"},
    mode="static",
)

# Create table with questions
df = pd.DataFrame(
    {
        "query": [
            "When it is burned what does hydrogen make?",
            "What was undertaken in 2010 to determine where dogs originated from?",
        ]
    }
)
query = pw.debug.table_from_pandas(df)

In [7]:
# Index for finding closest documents
index = VectorDocumentIndex(
    documents.doc, documents, embedder, n_dimensions=embedding_dimension
)

In [8]:
# LLM
if run_local == "Yes":
    pass
    #llm = ChatOllama(model=local_llm, temperature=0)
else:
    model = LiteLLMChat(
        model="mistral/mistral-large-latest",  #mistral-medium didn't get the answers right / mistral-large-latest also not
        temperature=0, 
        api_key=mistral_api_key
    )

In [9]:
# Run Adaptive RAG
result = query.select(
    question=query.query,
    result=answer_with_geometric_rag_strategy_from_index(
        query.query,
        index,
        documents.doc,
        model,
        n_starting_documents=2, 
        factor=2,
        max_iterations=5, 
    ),
)

In [10]:
# change logging configuration. Uncomment this if you want to see requests to Mistral in the logs
import logging
import sys

logging.basicConfig(stream=sys.stderr, level=logging.INFO, force=True)

In [11]:
%%time

# uncomment this line if you want to run the calculations and print the result
pw.debug.compute_and_print(result)

# 3m8s for capacity=3
# 2m36s for capacity=5

INFO:pathway_engine.engine.dataflow:Preparing Pathway computation
INFO:pathway_engine.connectors.monitoring:FilesystemReader-0: 0 entries (1 minibatch(es)) have been sent to the engine
INFO:pathway_engine.connectors.monitoring:FilesystemReader-0: 1007 entries (2 minibatch(es)) have been sent to the engine
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HT


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /embe


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embedding

...

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
11:15:22 - LiteLLM:INFO: utils.py:1018 - 

POST Request Sent from LiteLLM:
curl -X POST \
https://api.mistral.ai/v1 \
-d '{'model': 'mistral-embed', 'input': ['Throughout the 17th century Plymouth had gradually lost its pre-eminence as a trading port. By the mid-17th century commodities manufactured elsewhere in England cost too much to transport to Plymouth and the city had no means of processing sugar or tobacco imports, although it did play a relatively small part in the Atlantic slave trade during the early 18th century.']}'


INFO:LiteLLM:

POST Request Sent from LiteLLM:
curl -X POST \
https://api.mistral.ai/v1 \
-d '{'model': 'mistral-embed', 'input': ['Throughout the 17th century Plymouth had gradually lost its pre-eminence as a trading port. By the mid-17th century commodities manufactured elsewhere in England cost too much to transport to Plymouth and the city had no means of processing sugar

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
11:15:23 - LiteLLM:INFO: utils.py:1018 - 

POST Request Sent from LiteLLM:
curl -X POST \
https://api.mistral.ai/v1 \
-d '{'model': 'mistral-embed', 'input': ['The 2014 Human Development Report by the United Nations Development Program was released on July 24, 2014, and calculates HDI values based on estimates for 2013. Below is the list of the "very high human development" countries:']}'


INFO:LiteLLM:

POST Request Sent from LiteLLM:
curl -X POST \
https://api.mistral.ai/v1 \
-d '{'model': 'mistral-embed', 'input': ['The 2014 Human Development Report by the United Nations Development Program was released on July 24, 2014, and calculates HDI values based on estimates for 2013. Below is the list of the "very high human development" countries:']}'


INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
11:15:23 - LiteLLM:INFO: utils.py:1018 - 

POST Request Sent from Lite

INFO:LiteLLM:

POST Request Sent from LiteLLM:
curl -X POST \
https://api.mistral.ai/v1 \
-d '{'model': 'mistral-embed', 'input': ['While historically, symbiosis has received less attention than other interactions such as predation or competition, it is increasingly recognized as an important selective force behind evolution, with many species having a long history of interdependent co-evolution. In fact, the evolution of all eukaryotes (plants, animals, fungi, and protists) is believed under the endosymbiotic theory to have resulted from a symbiosis between various sorts of bacteria. This theory is supported by certain organelles dividing independently of the cell, and the observation that some organelles seem to have their own nucleic acid.']}'


INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
11:15:25 - LiteLLM:INFO: utils.py:1018 - 

POST Request Sent from LiteLLM:
curl -X POST \
https://api.mistral.ai/v1 \
-d '{'model': 'mistral-embed', 'input'

INFO:LiteLLM:

POST Request Sent from LiteLLM:
curl -X POST \
https://api.mistral.ai/v1 \
-d '{'model': 'mistral-embed', 'input': ['In Croatia there are many polytechnic institutes and colleges that offer a polytechnic education. The law about polytechnic education in Croatia was passed in 1997.']}'


INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
11:15:26 - LiteLLM:INFO: utils.py:1018 - 

POST Request Sent from LiteLLM:
curl -X POST \
https://api.mistral.ai/v1 \
-d '{'model': 'mistral-embed', 'input': ['The Polytechnic institutes in Pakistan, offer a diploma spanning three years in different branches. Students are admitted to the diploma program based on their results in the 10th grade standardized exams. The main purpose of Polytechnic Institutes is to train people in various trades.']}'


INFO:LiteLLM:

POST Request Sent from LiteLLM:
curl -X POST \
https://api.mistral.ai/v1 \
-d '{'model': 'mistral-embed', 'input': ['The Polytechnic institutes 

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
11:15:27 - LiteLLM:INFO: utils.py:1018 - 

POST Request Sent from LiteLLM:
curl -X POST \
https://api.mistral.ai/v1 \
-d '{'model': 'mistral-embed', 'input': ['Color temperature for white light sources also affects their use for certain applications. The color temperature of a white light source is the temperature in Kelvin of a theoretical black body emitter that most closely matches the spectral characteristics of the lamp. An incandescent bulb has a color temperature around 2800 to 3000 Kelvin; daylight is around 6400 Kelvin. Lower color temperature lamps have relatively more energy in the yellow and red part of the visible spectrum, while high color temperatures correspond to lamps with more of a blue-white appearance. For critical inspection or color matching tasks, or for retail displays of food and clothing, the color temperature of the lamps will be selected for the best overall lighting effect

In [1]:
import getpass
import os

import pandas as pd

import pathway as pw
from pathway.stdlib.indexing import VectorDocumentIndex
from pathway.xpacks.llm.embedders import LiteLLMEmbedder, SentenceTransformerEmbedder #OpenAIEmbedder
from pathway.xpacks.llm.llms import LiteLLMChat #OpenAIChat 
from pathway.xpacks.llm.question_answering import (
    answer_with_geometric_rag_strategy_from_index,
)

In [12]:
# ## OPENAI

# from pathway.xpacks.llm.embedders import OpenAIEmbedder
# from pathway.xpacks.llm.llms import OpenAIChat 

# # Set OpenAI API Key
# if "OPENAI_API_KEY" in os.environ:
#     api_key = os.environ["OPENAI_API_KEY"]
# else:
#     api_key = getpass.getpass("OpenAI API Key:")

# # Parameters for OpenAI models
# embedder_locator: str = "text-embedding-ada-002"
# embedding_dimension: int = 1536
# chat_locator: str = "gpt-3.5-turbo-16k-0613"
# max_tokens: int = 120
# temperature: float = 0.0

# # Set up OpenAI Embedder and Chat
# embedder = OpenAIEmbedder(
#     api_key=api_key,
#     model=embedder_locator,
#     retry_strategy=pw.udfs.FixedDelayRetryStrategy(),
#     cache_strategy=pw.udfs.DefaultCache(),
# )

# model = OpenAIChat(
#     api_key=api_key,
#     model=chat_locator,
#     temperature=temperature,
#     max_tokens=max_tokens,
#     retry_strategy=pw.udfs.FixedDelayRetryStrategy(),
#     cache_strategy=pw.udfs.DefaultCache(),
# )

OpenAI API Key: ········


In [ ]:
# ## MISTRAL AI

# # Flags for running locally
# run_local = "No"
# local_model = "ollama/mistral:latest"

# # Check API key
# if "MISTRAL_API_KEY" in os.environ:
#     mistral_api_key = os.environ["MISTRAL_API_KEY"]
# else:
#     mistral_api_key = getpass.getpass("Mistral API Key:")

# # Embed and index
# embedding_dimension: int = 1024

# if run_local == "Yes":
#     embedder = SentenceTransformerEmbedder(model="intfloat/e5-large-v2")
# else:
#     embedder = LiteLLMEmbedder(
#         capacity = 5, 
#         retry_strategy = pw.udfs.FixedDelayRetryStrategy(),
#         model = "mistral/mistral-embed",
#         api_key=mistral_api_key,
#     )

# # LLM
# if run_local == "Yes":
#     model = LiteLLMChat(
#         model=local_model, 
#         temperature=0
#     )
# else:
#     model = LiteLLMChat(
#         model="mistral/mistral-large-latest",  #mistral-medium didn't get the answers right / mistral-large-latest also not
#         temperature=0, 
#         api_key=mistral_api_key
#     )

In [ ]:
## MISTRAL LOCAL
embedder = SentenceTransformerEmbedder(model="intfloat/e5-large-v2")

In [ ]:
model = LiteLLMChat(
    model="ollama/mistral:latest", 
    temperature=0
)

In [16]:
# Load documents in which answers will be searched
class InputSchema(pw.Schema):
    doc: str

documents = pw.io.fs.read(
    "adaptive-rag-contexts.jsonl",
    format="json",
    schema=InputSchema,
    json_field_paths={"doc": "/context"},
    mode="static",
)

# Create table with questions
df = pd.DataFrame(
    {
        "query": [
            "When it is burned what does hydrogen make?",
            "What was undertaken in 2010 to determine where dogs originated from?",
        ]
    }
)
query = pw.debug.table_from_pandas(df)

# Main part of the code - creating index of documents and running adaptive RAG!

# Index for finding closest documents
index = VectorDocumentIndex(
    documents.doc, documents, embedder, n_dimensions=embedding_dimension
)

# Run Adaptive RAG
result = query.select(
    question=query.query,
    result=answer_with_geometric_rag_strategy_from_index(
        query.query,
        index,
        documents.doc,
        model,
        n_starting_documents=2,
        factor=2,
        max_iterations=5,
    ),
)

In [17]:
%%time
# uncomment this line if you want to run the calculations and print the result
print(f"running with {model}")
pw.debug.compute_and_print(result)

running with <pathway.xpacks.llm.llms.LiteLLMChat object at 0x31758cd50>


INFO:pathway_engine.engine.dataflow:Preparing Pathway computation
INFO:pathway_engine.connectors.monitoring:FilesystemReader-0: 0 entries (1 minibatch(es)) have been sent to the engine
INFO:pathway_engine.connectors.monitoring:FilesystemReader-0: 1007 entries (2 minibatch(es)) have been sent to the engine
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 429 Too Many Requests"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /embe


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



...


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /embeddings in 0.869474 seconds
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request t


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /embeddings in 0.956010 seconds
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/em

            | question                                                             | result
^YYY4HAB... | What was undertaken in 2010 to determine where dogs originated from? | I could not find an answer. The sources provided do not mention any event or study conducted in 2010 to determine the origin of dogs.
^X1MXHYY... | When it is burned what does hydrogen make?                           | The provided sources do not mention what hydrogen produces when it is burned. I could not find an answer.
CPU times: user 40.9 s, sys: 3.45 s, total: 44.4 s
Wall time: 2min 43s


## Results

### OPEN AI
- wall time: 19.7s
- results:

```
            | question                                                             | result
^YYY4HAB... | What was undertaken in 2010 to determine where dogs originated from? | A study was undertaken in 2010 to determine where dogs originated from [I could not find an answer].
^X1MXHYY... | When it is burned what does hydrogen make?                           | When hydrogen is burned, it produces water [2].
```


### MISTRAL
- wall time: 2m43s
- results:


```
            | question                                                             | result
^YYY4HAB... | What was undertaken in 2010 to determine where dogs originated from? | I could not find an answer. The sources provided do not mention any event or study conducted in 2010 to determine the origin of dogs.
^X1MXHYY... | When it is burned what does hydrogen make?                           | The provided sources do not mention what hydrogen produces when it is burned. I could not find an answer.
```